In [1]:
#!pip install --upgrade pip
import re
import xml.sax.saxutils as saxutils

#from BeautifulSoup import BeautifulSoup
#!pip install gensim
from gensim.models.word2vec import Word2Vec
#!pip install keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, LSTM
from keras.layers import Convolution1D, Flatten, Dropout
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.callbacks import TensorBoard
from keras.layers import Conv2D, MaxPooling2D
#!pip install multiprocessing
from multiprocessing import cpu_count
#!pip install nltk
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer, sent_tokenize
from nltk.stem import WordNetLemmatizer
#!pip install pandas
import numpy as np
import pandas as pd

from pandas import DataFrame
#!pip install sklearn
from sklearn.cross_validation import train_test_split
from sklearn import preprocessing
from sklearn import cross_validation

from sklearn import metrics
from numpy import linalg
from math import sqrt

Using TensorFlow backend.
/opt/conda/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [20]:
df_data = pd.read_csv('data_tactic_pool/data_domain_tactic_vectors.txt.gz',delim_whitespace=True, compression='gzip', header=0)

X = df_data
print(X.shape)

Y = pd.read_csv('data_tactic_pool/data_domain_tactic_labels.txt',sep='\t', header=0)
print(Y.shape)


(172531, 100)
(172531, 1)


In [21]:
from keras.callbacks import Callback

class BatchLogger(Callback):
    def on_train_begin(self, epoch, logs={}):
        self.log_values = {}
        for k in self.params['metrics']:
            self.log_values[k] = []

    def on_epoch_end(self, batch, logs={}):
        for k in self.params['metrics']:
            if k in logs:
                self.log_values[k].append(logs[k])
    
    def get_values(self, metric_name, window):
        d =  pd.Series(self.log_values[metric_name])
        return d.rolling(window,center=False).mean()

bl = BatchLogger()

In [23]:
from sklearn.model_selection import StratifiedKFold
from sklearn.cross_validation import KFold
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
#X_train, X_test, Y_train, Y_test = train_test_split(X, df_with_dummies, test_size=0.3)

df_label_NM = Y.as_matrix()
df_label_NM_1D = df_label_NM.ravel()
df_data_NM = X.as_matrix()

#skf = cross_validation.StratifiedKFold(df_label_NM,n_folds=10)
skf = StratifiedKFold(n_splits=10)
kf = KFold(X.shape[0], n_folds=10)

fold_accuracy = []

fold_number = 1

#for train_index, test_index in skf:
#for train_index, test_index in skf.split(df_data_NM, df_label_NM):
for train_index, test_index in kf:
    #print("Fold Number:",fold_number)
    fold_number+=1
    x_train, x_test = df_data_NM[train_index], df_data_NM[test_index]
    y_train, y_test = df_label_NM_1D[train_index], df_label_NM_1D[test_index]
    
    #print(y_test[5])
    
    input_dim = x_train.shape[1]

    model = Sequential()
    model.add(Dense(256, input_shape=(input_dim,), activation='relu'))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(10, activation='sigmoid'))
    model.add(Dense(1,  activation='sigmoid'))

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    history = model.fit(
              np.array(x_train), np.array(y_train),
              batch_size=25, nb_epoch=100, verbose=1, callbacks=[bl],
              validation_data=(np.array(x_test), np.array(y_test)))
    
    score = model.evaluate(np.array(x_test), np.array(y_test), verbose=0)
    y_test_pred = model.predict_on_batch(np.array(x_test))[:,0]
    
    score_accracy = score[1]
    fold_accuracy.append(score_accracy)
    
    y_train_pred = model.predict_on_batch(np.array(x_train))[:,0]
    
print("Accuracy per fold: ", fold_accuracy)
print("Average Accuracy: ", sum(fold_accuracy)/len(fold_accuracy), "\n")




/opt/conda/lib/python3.6/site-packages/keras/models.py:844: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 155277 samples, validate on 17254 samples
Epoch 1/100
155277/155277 [==============================] - 29s - loss: 0.4863 - acc: 0.7844 - val_loss: 1.6905 - val_acc: 0.0505
Epoch 2/100
155277/155277 [==============================] - 28s - loss: 0.4274 - acc: 0.8089 - val_loss: 1.7213 - val_acc: 0.0805
Epoch 3/100
155277/155277 [==============================] - 28s - loss: 0.4027 - acc: 0.8206 - val_loss: 2.2583 - val_acc: 0.0472
Epoch 4/100
155277/155277 [==============================] - 29s - loss: 0.3855 - acc: 0.8286 - val_loss: 1.9382 - val_acc: 0.0966
Epoch 5/100
155277/155277 [==============================] - 29s - loss: 0.3719 - acc: 0.8359 - val_loss: 2.2218 - val_acc: 0.0974
Epoch 6/100
155277/155277 [==============================] - 28s - loss: 0.3600 - acc: 0.8420 - val_loss: 2.2281 - val_acc: 0.1203
Epoch 7/100
155277/155277 [==============================] - 29s - loss: 0.3485 - acc: 0.8474 - val_loss: 2.4517 - val_acc: 0.1150
Epoch 8/100
155277/155277 [=====

155277/155277 [==============================] - 27s - loss: 0.1344 - acc: 0.9463 - val_loss: 4.2550 - val_acc: 0.1670
Epoch 63/100
155277/155277 [==============================] - 27s - loss: 0.1332 - acc: 0.9467 - val_loss: 4.1628 - val_acc: 0.1691
Epoch 64/100
155277/155277 [==============================] - 27s - loss: 0.1301 - acc: 0.9482 - val_loss: 4.2003 - val_acc: 0.1599
Epoch 65/100
155277/155277 [==============================] - 27s - loss: 0.1294 - acc: 0.9484 - val_loss: 3.9403 - val_acc: 0.1900
Epoch 66/100
155277/155277 [==============================] - 27s - loss: 0.1296 - acc: 0.9483 - val_loss: 3.8580 - val_acc: 0.1924
Epoch 67/100
155277/155277 [==============================] - 27s - loss: 0.1291 - acc: 0.9484 - val_loss: 3.9661 - val_acc: 0.1641
Epoch 68/100
155277/155277 [==============================] - 27s - loss: 0.1276 - acc: 0.9489 - val_loss: 4.5434 - val_acc: 0.1596
Epoch 69/100
155277/155277 [==============================] - 27s - loss: 0.1282 - acc: 0

155278/155278 [==============================] - 28s - loss: 0.2826 - acc: 0.8778 - val_loss: 2.0646 - val_acc: 0.2208
Epoch 24/100
155278/155278 [==============================] - 27s - loss: 0.2766 - acc: 0.8802 - val_loss: 2.2057 - val_acc: 0.2648
Epoch 25/100
155278/155278 [==============================] - 28s - loss: 0.2726 - acc: 0.8823 - val_loss: 2.1285 - val_acc: 0.2915
Epoch 26/100
155278/155278 [==============================] - 27s - loss: 0.2690 - acc: 0.8842 - val_loss: 2.3102 - val_acc: 0.2482
Epoch 27/100
155278/155278 [==============================] - 26s - loss: 0.2636 - acc: 0.8856 - val_loss: 2.1109 - val_acc: 0.2602
Epoch 28/100
155278/155278 [==============================] - 26s - loss: 0.2581 - acc: 0.8883 - val_loss: 2.5614 - val_acc: 0.2112
Epoch 29/100
155278/155278 [==============================] - 26s - loss: 0.2534 - acc: 0.8913 - val_loss: 2.3220 - val_acc: 0.2602
Epoch 30/100
155278/155278 [==============================] - 26s - loss: 0.2501 - acc: 0

155278/155278 [==============================] - 26s - loss: 0.1446 - acc: 0.9414 - val_loss: 3.0361 - val_acc: 0.2757
Epoch 86/100
155278/155278 [==============================] - 26s - loss: 0.1439 - acc: 0.9421 - val_loss: 3.0744 - val_acc: 0.2858
Epoch 87/100
155278/155278 [==============================] - 26s - loss: 0.1441 - acc: 0.9415 - val_loss: 3.0911 - val_acc: 0.2644
Epoch 88/100
155278/155278 [==============================] - 26s - loss: 0.1428 - acc: 0.9425 - val_loss: 3.0748 - val_acc: 0.2739
Epoch 89/100
155278/155278 [==============================] - 26s - loss: 0.1430 - acc: 0.9429 - val_loss: 3.1155 - val_acc: 0.2707
Epoch 90/100
155278/155278 [==============================] - 28s - loss: 0.1426 - acc: 0.9430 - val_loss: 2.9864 - val_acc: 0.2991
Epoch 91/100
155278/155278 [==============================] - 28s - loss: 0.1432 - acc: 0.9425 - val_loss: 2.9443 - val_acc: 0.2902
Epoch 92/100
155278/155278 [==============================] - 27s - loss: 0.1368 - acc: 0

...

155278/155278 [==============================] - 28s - loss: 0.2418 - acc: 0.8887 - val_loss: 1.2687 - val_acc: 0.6391
Epoch 38/100
155278/155278 [==============================] - 27s - loss: 0.2388 - acc: 0.8899 - val_loss: 1.4209 - val_acc: 0.5959
Epoch 39/100
155278/155278 [==============================] - 27s - loss: 0.2344 - acc: 0.8925 - val_loss: 1.3419 - val_acc: 0.6124
Epoch 40/100
155278/155278 [==============================] - 27s - loss: 0.2330 - acc: 0.8934 - val_loss: 1.2612 - val_acc: 0.6569
Epoch 41/100
155278/155278 [==============================] - 27s - loss: 0.2295 - acc: 0.8950 - val_loss: 1.2334 - val_acc: 0.6580
Epoch 42/100
155278/155278 [==============================] - 27s - loss: 0.2262 - acc: 0.8964 - val_loss: 1.2970 - val_acc: 0.6464
Epoch 43/100
155278/155278 [==============================] - 27s - loss: 0.2241 - acc: 0.8985 - val_loss: 1.2857 - val_acc: 0.6311
Epoch 44/100
155278/155278 [==============================] - 27s - loss: 0.2205 - acc: 0

155278/155278 [==============================] - 27s - loss: 0.1376 - acc: 0.9411 - val_loss: 1.6678 - val_acc: 0.6033
Epoch 100/100
155278/155278 [==============================] - 27s - loss: 0.1382 - acc: 0.9410 - val_loss: 1.7961 - val_acc: 0.5910
Train on 155278 samples, validate on 17253 samples
Epoch 1/100
155278/155278 [==============================] - 27s - loss: 0.5860 - acc: 0.6864 - val_loss: 0.3141 - val_acc: 0.9426
Epoch 2/100
155278/155278 [==============================] - 27s - loss: 0.5267 - acc: 0.7292 - val_loss: 0.4350 - val_acc: 0.8767
Epoch 3/100
155278/155278 [==============================] - 27s - loss: 0.5040 - acc: 0.7430 - val_loss: 0.4004 - val_acc: 0.8724
Epoch 4/100
155278/155278 [==============================] - 28s - loss: 0.4871 - acc: 0.7517 - val_loss: 0.6769 - val_acc: 0.6646
Epoch 5/100
155278/155278 [==============================] - 28s - loss: 0.4732 - acc: 0.7612 - val_loss: 0.5833 - val_acc: 0.7273
Epoch 6/100
155278/155278 [===============

155278/155278 [==============================] - 29s - loss: 0.1981 - acc: 0.9122 - val_loss: 0.9366 - val_acc: 0.7245
Epoch 61/100
155278/155278 [==============================] - 28s - loss: 0.1966 - acc: 0.9123 - val_loss: 0.8323 - val_acc: 0.7478
Epoch 62/100
155278/155278 [==============================] - 28s - loss: 0.1951 - acc: 0.9129 - val_loss: 1.0386 - val_acc: 0.6628
Epoch 63/100
155278/155278 [==============================] - 28s - loss: 0.1963 - acc: 0.9140 - val_loss: 0.9487 - val_acc: 0.7210
Epoch 64/100
155278/155278 [==============================] - 28s - loss: 0.1918 - acc: 0.9149 - val_loss: 0.8879 - val_acc: 0.7214
Epoch 65/100
155278/155278 [==============================] - 28s - loss: 0.1906 - acc: 0.9167 - val_loss: 0.8791 - val_acc: 0.7435
Epoch 66/100
155278/155278 [==============================] - 28s - loss: 0.1910 - acc: 0.9158 - val_loss: 0.9244 - val_acc: 0.7063
Epoch 67/100
155278/155278 [==============================] - 29s - loss: 0.1880 - acc: 0

155278/155278 [==============================] - 29s - loss: 0.3386 - acc: 0.8412 - val_loss: 0.5257 - val_acc: 0.8105
Epoch 22/100
155278/155278 [==============================] - 29s - loss: 0.3321 - acc: 0.8448 - val_loss: 0.6446 - val_acc: 0.7624
Epoch 23/100
155278/155278 [==============================] - 29s - loss: 0.3265 - acc: 0.8478 - val_loss: 0.6281 - val_acc: 0.7803
Epoch 24/100
155278/155278 [==============================] - 28s - loss: 0.3195 - acc: 0.8518 - val_loss: 0.7141 - val_acc: 0.7233
Epoch 25/100
155278/155278 [==============================] - 29s - loss: 0.3142 - acc: 0.8550 - val_loss: 0.5422 - val_acc: 0.8202
Epoch 26/100
155278/155278 [==============================] - 29s - loss: 0.3076 - acc: 0.8581 - val_loss: 0.7311 - val_acc: 0.7460
Epoch 27/100
155278/155278 [==============================] - 29s - loss: 0.3039 - acc: 0.8603 - val_loss: 0.7969 - val_acc: 0.6900
Epoch 28/100
155278/155278 [==============================] - 29s - loss: 0.2980 - acc: 0

155278/155278 [==============================] - 28s - loss: 0.1676 - acc: 0.9288 - val_loss: 0.9603 - val_acc: 0.7158
Epoch 84/100
155278/155278 [==============================] - 28s - loss: 0.1675 - acc: 0.9281 - val_loss: 0.9282 - val_acc: 0.7184
Epoch 85/100
155278/155278 [==============================] - 28s - loss: 0.1656 - acc: 0.9299 - val_loss: 1.0073 - val_acc: 0.6992
Epoch 86/100
155278/155278 [==============================] - 28s - loss: 0.1683 - acc: 0.9293 - val_loss: 0.8586 - val_acc: 0.7276
Epoch 87/100
155278/155278 [==============================] - 28s - loss: 0.1661 - acc: 0.9297 - val_loss: 0.9835 - val_acc: 0.7192
Epoch 88/100
155278/155278 [==============================] - 28s - loss: 0.1626 - acc: 0.9308 - val_loss: 0.9552 - val_acc: 0.7115
Epoch 89/100
155278/155278 [==============================] - 28s - loss: 0.1627 - acc: 0.9310 - val_loss: 1.0808 - val_acc: 0.6901
Epoch 90/100
155278/155278 [==============================] - 28s - loss: 0.1623 - acc: 0